# Random baseline

In [1]:
from datasets import load_from_disk
from src.utils.config_management import CONFIG

## Data loading

In [2]:
hf_dataset = load_from_disk(CONFIG['paths']['data']['dalip_hf_dataset'])

In [3]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 'question_view_count', 'question_body', 'question_last_edit_date', 'question_last_activity_date', 'question_title', 'question_tags', 'question_answer_count', 'question_comment_count', 'question_favorite_count', 'question_closed_date', 'question_community_owned_date', 'answer_accepted'],
        num_rows: 42700
    })
    test: Dataset({
        features: ['answer_id', 'question_id', 'answer_creation_date', 'answer_score', 'answer_normalized_score', 'answer_log_normalized_score', 'answer_body', 'answer_last_edit_date', 'answer_last_activity_date', 'answer_comment_count', 'answer_community_owned_date', 'question_creation_date', 'question_score', 

## Create predictions

In [4]:
import numpy as np

In [5]:
TARGET_COL = 'answer_normalized_score'

In [6]:
y_train = hf_dataset['train'][TARGET_COL]
y_test = hf_dataset['test'][TARGET_COL]

In [7]:
np.random.seed(42)
predictions = np.random.choice(y_train, len(y_test), replace=True)

## Evaluation

In [8]:
import pandas as pd
import wandb
from collections import defaultdict
from src.evaluation import RankingEvaluator

In [24]:
wandb_config = defaultdict(dict)

wandb_config['regression_model']['regressor_type'] = 'random sampling'

wandb_config

defaultdict(dict,
            {'regression_model': {'regressor_type': 'constant prediction'}})

In [25]:
run = wandb.init(
    project='dalip-stackoverflow-answer-ranking',
    config=wandb_config,
    tags=['random', 'baseline']
)

### Compute metrics

In [26]:
test_question_ids = hf_dataset['test']['question_id']

In [27]:
evaluator = RankingEvaluator(ndcg_k=list(range(1, 11)), 
                             ndcg_gain_func='exponential', ndcg_discount_func='logarithmic')

In [29]:
metrics = evaluator(y_test, predictions, test_question_ids)
if TARGET_COL == 'answer_log_normalized_score':
    metrics.pop('mae')
metrics

{'ndcg@1_g.exponential_d.logarithmic': 0.2967333440571086,
 'ndcg@2_g.exponential_d.logarithmic': 0.39102355292621294,
 'ndcg@3_g.exponential_d.logarithmic': 0.49051441038311916,
 'ndcg@4_g.exponential_d.logarithmic': 0.5935327457991202,
 'ndcg@5_g.exponential_d.logarithmic': 0.6363236504851059,
 'ndcg@6_g.exponential_d.logarithmic': 0.6547649174390316,
 'ndcg@7_g.exponential_d.logarithmic': 0.6637396364657419,
 'ndcg@8_g.exponential_d.logarithmic': 0.6686781532230255,
 'ndcg@9_g.exponential_d.logarithmic': 0.6722785237330156,
 'ndcg@10_g.exponential_d.logarithmic': 0.6737536205493905,
 'mae': 18.44135188866799,
 'hit_rate@1': 0.36349614395886887}

In [30]:
wandb.log(metrics)

### Log predictions

In [31]:
predictions_df = pd.DataFrame()
predictions_df['answer_id'] = hf_dataset['test']['answer_id']
predictions_df['answer_normalized_score'] = hf_dataset['test']['answer_normalized_score']
predictions_df['predicted_score'] = predictions

In [32]:
wandb.log({'predictions_table': wandb.Table(dataframe=predictions_df)})

In [33]:
run.finish()

hit_rate@1,▁
mae,▁
ndcg@10_g.exponential_d.logarithmic,▁
ndcg@1_g.exponential_d.logarithmic,▁
ndcg@2_g.exponential_d.logarithmic,▁
ndcg@3_g.exponential_d.logarithmic,▁
ndcg@4_g.exponential_d.logarithmic,▁
ndcg@5_g.exponential_d.logarithmic,▁
ndcg@6_g.exponential_d.logarithmic,▁
ndcg@7_g.exponential_d.logarithmic,▁
ndcg@8_g.exponential_d.logarithmic,▁
